In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableBranch

In [4]:
llm  = ChatOpenAI(
    model= "gpt-4o-mini", 
    temperature= 0, 
    top_p =  0.95, 
    frequency_penalty=1.2
)

In [5]:
#Sub agent handlers
def booking_handler(request:str) :
    print("Delegating to booking handler")
    return f"Booking Handler processed request : {request}. Result : Simulated booking action."

def info_handler(request:str) :
    print("Delegating to Info handler")
    return f"Info Handler processed request : {request}. Result : Simulated information retrieval."

def unclear_handler(request:str) :
    print("Delegating to unclear handler")
    return f"Coordinator could not delegate request : {request}. Please clarify."
    

In [6]:
#Here we are using llm routing
router_prompt =  ChatPromptTemplate.from_messages(
    [
        ("system",
        """Analyse the user's request and determine which specialist handler should procees it. 
        - If the request is related to booking flights or hotels, output 'booker'.
        - For all other general information questions, output 'info'.
        - If the request is unclear or does not fit either category, output 'unclear'.
        ONLY output one word : 'booker', 'info' or 'unclear'.
        """),
        ("user","{request}")
    ]
)

In [7]:
router_chain = router_prompt | llm | StrOutputParser()

In [8]:
branches = {
    "booker" : RunnablePassthrough.assign(output = lambda x : booking_handler(x["request"]["request"])),
    "info" : RunnablePassthrough.assign(output = lambda x : info_handler(x["request"]["request"])),
    "unclear" : RunnablePassthrough.assign(output = lambda x : unclear_handler(x["request"]["request"]))
}

In [19]:
delegation_branch = RunnableBranch(
    (lambda x : x["decision"].strip() == 'booker' , branches["booker"]),
    (lambda x : x["decision"].strip() == "info", branches["info"]),
    branches["unclear"]
)

In [20]:
agent = {
    "decision" : router_chain,
    "request" : RunnablePassthrough()
} | delegation_branch | (lambda x : x["output"])

In [21]:
def main():
    # request_a = "Book me a flight to London."
    # result_a = agent.invoke({"request":request_a})
    # print(result_a)
    
    request_b = "What is the capital of Italy?"
    result_b = agent.invoke({"request":request_b})
    print(result_b)
    
main()

Delegating to Info handler
Info Handler processed request : What is the capital of Italy?. Result : Simulated information retrieval.
